In [3]:
import numpy as np
import pandas as pd
import time

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.metrics import roc_auc_score

import xgboost as xgb
import lightgbm as lgb
from lightgbm import log_evaluation, early_stopping


# jupyter配置
pd.options.display.max_rows=1000 #Notebook 的一个cell的显示行数
pd.options.display.max_columns=10000#Notebook 的一个cell的显示列数

In [31]:
train_data = pd.read_csv('./data/train.csv')
train_data_y= train_data.loc[:, 'rating']
train_data_x = train_data.loc[:, ["user_id", "product_id", "votes", "helpful_votes"]] # "product_name", 
test_data = pd.read_csv('./data/test.csv')
test_data = test_data.loc[:, ["user_id", "product_id"]]
X_train, X_test, y_train, y_test = train_test_split(train_data_x, train_data_y, test_size=0.3)


In [33]:
print(train_data.shape, X_train.shape, X_test.shape, y_train.shape, y_test.shape, test_data.shape)

(745889, 7) (522122, 4) (223767, 4) (522122,) (223767,) (223553, 2)


In [34]:
train_data_x.describe()
train_data_x.dtypes
test_data.dtypes

user_id       int64
product_id    int64
dtype: object

In [36]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
test_x_to_predict = xgb.DMatrix(test_data)

# 设置参数
params = {
    'objective': 'multi:softmax',
    'num_class': 5,
    'eta': 0.1,
    'reg_alpha': 0.01,
    'reg_lambda': 0.01,
    'max_depth': 8,
    "enable_categorical" : True
}

# 训练模型
bst = xgb.train(
    params=params,
    dtrain=dtrain,
    early_stopping_rounds=10,
    num_boost_round=200,
    evals=[(dtrain, 'train'), (dtest, 'test')] # 将训练数据和测试数据都作为验证集，可以实时监督训练情况，是否过拟合
)

# 预测结果
result = bst.predict(
    dtest
)
print('Accuracy of prediction on dataset:', accuracy_score(y_test, result))

# 提交submit
model = bst
res = model.predict(test_x_to_predict)
# res = np.array([np.argmax(l) for l in res])
submit = pd.DataFrame({'id':range(len(res)), 'label':res}).astype('int32')
submit.to_csv(f"./data/submit_example_A{time.strftime('%Y%m%d%H%M', time.localtime())}.csv",index=False, encoding='utf-8')

/Users/wzq/anaconda3/envs/torch/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [16:47:45] WARNING: /var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_d9k8pmaj4_/croot/xgboost-split_1724073758172/work/src/learner.cc:740: 
Parameters: { "enable_categorical" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoostError: [16:47:45] /var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_d9k8pmaj4_/croot/xgboost-split_1724073758172/work/src/objective/multiclass_obj.cu:111: SoftmaxMultiClassObj: label must be in [0, num_class).
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x00000002a06db1f0 dmlc::LogMessageFatal::~LogMessageFatal() + 124
  [bt] (1) 2   libxgboost.dylib                    0x00000002a096021c xgboost::obj::SoftmaxMultiClassObj::GetGradient(xgboost::HostDeviceVector<float> const&, xgboost::MetaInfo const&, int, xgboost::linalg::Tensor<xgboost::detail::GradientPairInternal<float>, 2>*) + 708
  [bt] (2) 3   libxgboost.dylib                    0x00000002a08d53bc xgboost::LearnerImpl::UpdateOneIter(int, std::__1::shared_ptr<xgboost::DMatrix>) + 668
  [bt] (3) 4   libxgboost.dylib                    0x00000002a06fe288 XGBoosterUpdateOneIter + 160
  [bt] (4) 5   libffi.8.dylib                      0x000000010338004c ffi_call_SYSV + 76
  [bt] (5) 6   libffi.8.dylib    

In [14]:
train_x

,user_id,product_id,product_name,votes,helpful_votes
0,1813,154533,Beautiful Thing,10,8
1,1944,192838,Almost Famous,4,2
2,534,202590,A Clockwork Orange,5,5
3,1811,140456,Great Expectations (Wordsworth Classics),1,0
4,102,154278,Phenomenon,0,0
...,...,...,...,...,...
745884,1853,25651,"Star by Star (Star Wars: The New Jedi Order, B...",1,1
745885,1556,19657,"NIV Study Bible, Personal Size Indexed",13,11
745886,1737,157371,An Ideal Husband,1,1
745887,1916,24941,The Autobiography of Malcolm X (As Told to Ale...,1,1
